# Milestone Project 1: Food Vision Big


In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-a7ba7d26-5f3c-76a2-a07b-68784f9a8035)
GPU 1: Tesla T4 (UUID: GPU-3bd21e00-fa63-79b3-eae8-6003a2fffca4)


In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/refs/heads/main/extras/helper_functions.py

--2025-10-23 11:47:50--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/refs/heads/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0.001s  

2025-10-23 11:47:50 (14.7 MB/s) - ‘helper_functions.py’ saved [10246/10246]

